In [1]:
from Exercise import Exercise, MarkdownBlock
from config import URL, TOKEN
import json

import numpy as np
import sympy as sp

Exercise.URL = URL
Exercise.TOKEN = TOKEN

In [ ]:
from sympy import Rational, Symbol, latex, UnevaluatedExpr

u = lambda x : UnevaluatedExpr(x)

# Helper functions
def explain_add(a, b):
    assert(np.shape(a) == np.shape(b))
    rows, columns = np.shape(a)
    return sp.Matrix([[Symbol(f"({latex(u(a[i,j]))} + {latex(u(b[i,j]))})") for j in range(columns)] for i in range(rows)])

def symbolic_matrix(character, rows, columns):
    return sp.Matrix([[Symbol(f"{{{character}}}_{{{i+1}, {j+1}}}") for j in range(columns)] for i in range(rows)])

def explain_multiply(a, b):
    # #rows in b == #columns in a
    assert(np.shape(a)[1] == np.shape(b)[0])
    rows = np.shape(a)[0]
    columns = np.shape(b)[1]
    result = np.empty(shape=(rows, columns), dtype=object)
    for i in range(rows):
        row = a[i,:]
        for j in range(columns):
            column = b[:,j]
            zipped = zip(row, column)
            mapped = list(map(lambda t: f"{latex(u(t[0]))} \cdot {latex(u(t[1]))}", zipped))
            s = Symbol("") 
            result[i, j] = Symbol(" + ".join(mapped), evaluate=False)
                
    return sp.Matrix(result)

# TODO: fixme for >= 3 dim matrices
# TODO: print intermediate steps at each return
def explain_det(a):
    # square matrix
    assert(np.shape(a)[0] == np.shape(a)[1])
    if np.shape(a)[0] == 1:
        return a[0,0]
    if np.shape(a)[0] == 2:
        return f"{a[0,0]} \cdot {a[1,1]} - {a[0,1]} \cdot {a[0,1]}"
    else:
        return f"{a[0,0]} \cdot {explain_det(a[1:,1:])} - {a[0,1]} \cdot {[[a[1,0], a[1,2]], [a[2,0], a[2,2]]]} + {a[0,2]} \cdot {[[a[1,0], a[1,2]], [a[1,1], a[2,1]]]}"

# A = symbolic_matrix("a", 3, 3)
# e = Exercise(f"${explain_det(A)}$")
# e.display()

In [ ]:
def generator():
    length = np.random.randint(1, 7)
    v1 = sp.randMatrix(r=length, c=1, min=0, max=10)
    v2 = sp.randMatrix(r=length, c=1, min=0, max=10)

    s = "What is $@v1 + @v2$?"

    params = {}
    params["v1"] = v1 
    params["v2"] = v2 
    e = Exercise(MarkdownBlock(s, params))
    
    params["v3"] = explain_add(v1,v2)
    params["v4"] = v1 + v2 
    s1 = "Yes, $@v1 + @v2 = @v3 = @v4$!"
    e.add_answer(v1 + v2, True, MarkdownBlock(s1, params))

    a = symbolic_matrix("a", length, 1)
    b = symbolic_matrix("b", length, 1)
    ab = explain_add(a, b)
    default_feedback = "Remember the definition of matrix addition: $@a + @b = @ab$"
    e.add_default_feedback(MarkdownBlock(default_feedback, dict(a=a, b=b, ab=ab)))
    
    return e

Exercise.write_multiple(generator, 100, "vector_add_2")

In [1]:
s = "What is $@a @b$?"

rows = np.random.randint(1, 4)
columns = np.random.randint(1, 4)

params = {}
params["a"] = sp.randMatrix(r=rows, c=columns, min=0, max=10)
params["b"] = sp.randMatrix(r=columns, c=rows+2, min=0, max=10)
ans = params["a"] * params["b"]

e = Exercise(MarkdownBlock(s, params))

e.add_answer(ans, True, "That's right!")

e.play()

NameError: name 'np' is not defined

In [ ]:
def generator():
    s = "What is $@a @b$?"

    rows = np.random.randint(1, 4)
    columns = np.random.randint(1, 4)

    params = {}
    params["a"] = sp.randMatrix(r=rows, c=columns, min=0, max=10)
    params["b"] = sp.randMatrix(r=columns, c=rows+2, min=0, max=10)
    ans = params["a"] * params["b"]

    e = Exercise(MarkdownBlock(s, params))

    e.add_answer(ans, True, "That's right!")
    
    return e

Exercise.write_multiple(generator, 10, "mat_multiply")

In [ ]:
def generator():
    s = "What is $@a \cdot @b$?"

    rows = np.random.randint(1, 4)
    columns = np.random.randint(1, 4)

    params = {}
    params["a"] = sp.Matrix(np.random.randint(5, size=rows*columns).reshape((rows,columns)))
    params["b"] = sp.Matrix(np.random.randint(5, size=(2+rows)*columns).reshape((columns,rows+2)))

    e = Exercise(MarkdownBlock(s, params))

    ans = params["a"] * params["b"]
    e.add_answer(params["a"] * params["b"], True, "That's right!")

    params = {}
    params["x"] = symbolic_matrix("a", rows, columns)
    params["y"] = symbolic_matrix("b", columns, rows+2)
    params["z"] = explain_multiply(params["x"], params["y"])
    f = """Remember the definition of matrix multiplication: \n $@x \cdot @y = @z$"""
    e.add_default_feedback(MarkdownBlock(f, params))
    
    return e

Exercise.write_multiple(generator, 10, "mat_multiply")

In [ ]:
def explain_multiply(a, b):
    # #rows in b == #columns in a
    assert(np.shape(a)[1] == np.shape(b)[0])
    rows = np.shape(a)[0]
    columns = np.shape(b)[1]
    result = np.empty(shape=(rows, columns), dtype=object)
    for i in range(rows):
        row = a[i,:]
        for j in range(columns):
            column = b[:,j]
            zipped = zip(row, column)
            mapped = list(map(lambda t: f"{latex(u(t[0]))} \cdot {latex(u(t[1]))}", zipped))
            s = Symbol("") 
            result[i, j] = Symbol(" + ".join(mapped), evaluate=False)
                
    return sp.Matrix(result)

# explain_multiply(np.arange(4).reshape((2,2)), np.arange(4).reshape((2,2)))

In [ ]:
v = symbolic_matrix("a", 3, 1)
display(3*u(v))
display(3*v)


# TODO: meaningfull, contextualized exercises for each of these:
- scalar def.
- vector def.
- matrix def.
- vector indexing
- matrix indexing
- total values in a matrix
- vector dimensions
- matrix dimensions
- special matrices
- transpose
- operations (+, *), linear combinations
- dot product, cross product, norm
- cosine similarity

## Matrix Transpose

In [ ]:
m = "Determine $@a^\intercal$?"

a = np.arange(6).reshape((2, 3))

params = {}
params["a"] = sp.Matrix(a)

e = Exercise(MarkdownBlock(m, params))
e.display()

e.add_answer(params["a"].T, True, "Correct!")

e.write()
e.publish()

## Matrix Inverse

In [ ]:
m = "Determine $@a^{-1}$?"

a = np.arange(4).reshape((2, 2))

params = {}
params["a"] = sp.Matrix(a)

e = Exercise(MarkdownBlock(m, params))
e.display()

e.add_answer(params["a"].inv(), True, "Correct!")

e.write()
e.publish()

In [ ]:
m = "Given $A = @a$, what is $det(A)$?"

a = np.arange(4).reshape((2, 2))
params = {}
params["a"] = sp.Matrix(a)

e = Exercise(MarkdownBlock(m, params))
e.display()

e.add_answer(params["a"].det(), True, "Correct!")

e.write()
e.publish()